This notebook shows how to access data from the [AICC](http://fire.ak.blm.gov/predsvcs/maps.php)[Current Fire Data](http://afs.ak.blm.gov/arcgis/rest/services/MapAndFeatureServices/CurrentFiresService/MapServer) [ESRI REST services](http://resources.arcgis.com/en/help/rest/apiref/).  The goal is to access the data pertaining to the fire perimeters and the centroid point locations of all fires of the season (2016) and the current active fires.


#### install dependencies:

    pip install requests
    

#### Read in the FireData class:

In [1]:
class FireData( object ):
	def __init__( self, baseurl, *args, **kwargs ):
		'''
		return fire data from ESRI REST services at AICC

		ARGUMENTS:
		----------
		baseurl = [str] url to the base ESRI REST service.

		** for AICC holdings an example is as follows:
		'http://afs.ak.blm.gov/arcgis/rest/services/MapAndFeatureServices/\n
					CurrentFiresService/MapServer'
		RETURNS:
		--------
		object of type FireData containing the following attributes:
		[ baseurl, layers, queryurl, dat, fields, meta, json ]

		'''
		self.baseurl = baseurl
		self.layers = self._discover_layers()
		self.queryurl = None
		self.dat = None
		self.fields = None
		self.meta = None
		self.json = None

	def _discover_layers( self ):
		''' 
		return a dict of the available layers and their identifiers.
		this is useful for the discovery of what layer id's to use in
		FireData.get_data( )
		'''
		r = requests.get( self.baseurl + '/'+ 'layers', params={'f':'pjson'} )
		return { i['id']:i['name'] for i in r.json()[ 'layers' ] }
	def get_meta( self, layerid=0, params={'f':'pjson'} ):
		'''
		return layer metadata including FIELDS which is useful for 
		crafting the `outFields` {'key':'value'} pair which will tell
		the service what fields to return. There can be an abundance.

		ARGUMENTS:
		----------
		layerid = [int] id of the layer to select from. default:0
		params = [dict] default:{'f':'pjson'} which tells to 
					return as json.  

		RETURNS:
		--------
		self.meta and self.fields, where self.meta has all metadata as json
		about the layer being queried and self fields containing a list of dicts
		containing attributes of the fields.

		'''
		path = '/'.join([ self.baseurl, str( layerid ) ])
		r = requests.get( path,  params=params )
		self.meta = r.json()
		self.fields = r.json()[ 'fields' ]
	def get_data( self, layerid=0, params={'where':''}, crs=3572 ):
		'''
		make a request to the AICC REST services with a query dict of 
		{key:value} pairs of search parameters.  

		ARGUMENTS:
		----------
		layerid = [int] id of the layer to select from
		params = [dict] dict of {'key':'value'} pairs of search parameters
					to pass to the REST service.
		crs = [int] EPSG code for the reference system to return the data in.

		RETURNS:
		--------
		self.dat & self.json where .dat is the returned `requests` get object
		and .json is the dat.json() helper.

		'''
		path = '/'.join([ self.baseurl, str( layerid ), 'query' ])
		r = requests.get( path, params=params )
		self.dat = r
		self.json = self.dat.json()

In [2]:
import requests, subprocess, json

Setup variables for the baseurl of the mapserver.
The ESRI REST url standard for the service: `http://<catalog-url>/<serviceName>/MapServer`

In [3]:
baseurl = 'http://afs.ak.blm.gov/arcgis/rest/services/MapAndFeatureServices/CurrentFiresService/MapServer'

following the [available REST search parameters](http://resources.arcgis.com/en/help/rest/apiref/query.html) provided by the ESRI service, we build a dictionary of {key:value} pairs for each.  We only really *need* to give a `'where'` and the `'f':'pjson'` to tell the service what to grab and how to return the results.  

Many of these options that are not being used can be left out of this dictionary, as they have defaults, or they default to None and are not used in the query. 

In [4]:
QUERY = {'where' : '',
        'text' : '',
        'objectIds' : '',
        'time' : '',
        'geometry' : '',
        'geometryType' : 'esriGeometryEnvelope',
        'inSR' : '',
        'spatialRel' : 'esriSpatialRelIntersects',
        'relationParam' : '',
        'outFields' : '*',
        'returnGeometry' : 'true',
        'maxAllowableOffset' : '',
        'geometryPrecision' : '',
        'outSR' : '',
        'returnIdsOnly' : 'false',
        'returnCountOnly' : 'false',
        'orderByFields' : '',
        'groupByFieldsForStatistics' : '',
        'outStatistics' : '',
        'returnZ' : 'false',
        'returnM' : 'false',
        'gdbVersion' : '',
        'returnDistinctValues' : 'false',
        'returnTrueCurves' : 'false',
        'resultOffset' : '',
        'resultRecordCount' : '',
        'f' : 'pjson' }


Now lets instantiate the `FireData` object using the `baseurl` variable from above

In [5]:
f = FireData( baseurl )

What layers are available from this service and what are the numeric identifiers?

the following method call returns a `dict` of `int` identifiers of the layers as the keys and the string names as the values.

In [6]:
f.layers

{0: u'Active Fires',
 1: u'Active Fire Perimeters',
 2: u'All of This Years Fires',
 3: u'All of This Years Fire Perimeters'}

In this example we want to return all of the available fire perimeters for this season.  For some reason as of late, there are no active fire perimeters available through these queries.  There _are_ some small polygon perimeters returned from the `All of This Years Fire Perimeters` layer and I am going to query for these in hopes that new data begin to populate this server.

Now that we know what layer we want, lets set a `layerid` variable with the `int` key for that layer

In [7]:
layerid = 3

Lets get the metadata related to the layer we are interested in which also has information about the fields available for query and for returning with the returned JSON object.  

In [8]:
# this will populate the f.meta attribute of the current FireData object
f.get_meta( layerid=layerid )

# print out the name key for each of the fields elements in the returned list.
print [ i['name'] for i in f.meta['fields'] ]


[u'FIRESADMIN.Fire_perimeters.OBJECTID', u'FIRESADMIN.Fire_perimeters.NAME', u'FIRESADMIN.Fire_perimeters.RECORDNUMBER', u'FIRESADMIN.Fire_perimeters.ACRES', u'FIRESADMIN.Fire_perimeters.AFSNUMBER', u'FIRESADMIN.Fire_perimeters.DOFNUMBER', u'FIRESADMIN.Fire_perimeters.USFSNUMBER', u'FIRESADMIN.Fire_perimeters.ADDNUMBER', u'FIRESADMIN.Fire_perimeters.PERIMETERDATE', u'FIRESADMIN.Fire_perimeters.LATESTPERIMETER', u'FIRESADMIN.Fire_perimeters.SOURCE', u'FIRESADMIN.Fire_perimeters.SOURCEMETHOD', u'FIRESADMIN.Fire_perimeters.SOURCECLASS', u'FIRESADMIN.Fire_perimeters.AGENCYACRES', u'FIRESADMIN.Fire_perimeters.ACREAGEMETHOD', u'FIRESADMIN.Fire_perimeters.COMMENTS', u'FIRESADMIN.Fire_perimeters.FIREID', u'FIRESADMIN.Fire_perimeters.FIREYEAR', u'FIRESADMIN.Fire_perimeters.UPDATETIME', u'FIRESADMIN.Fire_perimeters.UPDATEUSER', u'FIRESADMIN.Fire_perimeters.USEDONFINALREPORT', u'FIRESADMIN.Fire_perimeters.SHAPE', u'SHAPE.AREA', u'SHAPE.LEN', u'FIRESADMIN.Fire.OBJECTID', u'FIRESADMIN.Fire.ID', u'F

Now we know the available fields and that there is one with the name `'FIRESADMIN.Fire.FIRESEASON'` which we can use to query for this years perimeters.


In [14]:
whereclause = "FIRESADMIN.Fire_perimeters.FIREYEAR='2016'"

# add it to the QUERY dict using update
QUERY.update( whereclause=whereclause )

now lets return the JSON we want relating to this query.

In [10]:
f.get_data( layerid, QUERY )

# write the json to disk
with open( './this_years_perimeters.json', 'w' ) as out_json:
    out_json.write( json.dumps( f.json ) )


In [11]:
# OR we can use the url generated by requests to issue a system command using subprocess
output_filename = out_json.name.replace( '.json', '.shp' )
_ = subprocess.call([ 'ogr2ogr', '-overwrite', output_filename, out_json.name ])

It is important to note that due to the very long field names, ogr2ogr has truncated field names and added a sequential number to make each field unique.  This is not so useful, so we should try to mess with that a bit more for a clean shapefile output.
